In [4]:
##%overwritefile
##%file:src/compile_with_javac.py
##%noruncode
    def compile_with_javac(self, source_filename, 
        binary_filepath=None, cflags=None, 
        ldflags=None,env=None,
        coptions=None,magics=None):
        # cflags = ['-std=c89', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        outpath=os.path.dirname(source_filename)
        sf = os.path.basename(source_filename)
        binary_filename = sf.split(".")[0]
        # if self.linkMaths:
        #     cflags = cflags + ['-lm']
        # if self.wError:
        #     cflags = cflags + ['-Werror']
        # if self.wAll:
        #     cflags = cflags + ['-Wall']
        # if self.readOnlyFileSystem:
        #     cflags = ['-DREAD_ONLY_FILE_SYSTEM'] + cflags
        # if self.bufferedOutput:
        #     cflags = ['-DBUFFERED_OUTPUT'] + cflags
        # target=os.path.dirname(binary_filepath)
        index=-1
        if coptions==None:
            coptions=[]
        for s in coptions:
            index=index+1
            if s=='-d':
                outpath=coptions[index+1]
                if not outpath.startswith('-'):
                    #剔除 -d参数和值
                    outpath=coptions.pop(index+1)
                    coptions.pop(index)
            else:
                if binary_filepath!=None:
                    outpath=binary_filepath
        args = ['javac']+coptions+ ['-d', outpath]+[ source_filename]
        self.mymagics._logln(' '.join((' '+ str(s) for s in args)))
        binary_filename=os.path.join(outpath,binary_filename)
        return self.mymagics.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename+".class",args


[MyPythonKernel125439] Info:src/compile_with_javac.py
[MyPythonKernel125439] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/Java/src/compile_with_javac.py created successfully


In [5]:
##调用 javac 编译源代码
##%overwritefile
##%file:src/j_exec_javac_.py
##%noruncode
    def _exec_javac_(self,source_filename,magics):
        self.mymagics._logln('Generating binary file')
        magics['status']='compiling'
        p,outfile,ccmd = self.compile_with_javac(
            source_filename, 
            None,
            self.mymagics.get_magicsSvalue(magics,'cflags'),
            self.mymagics.get_magicsSvalue(magics,'ldflags'),
            self.mymagics.get_magicsbykey(magics,'env'),
            self.mymagics.get_magicsSvalue(magics,'coptions'),
            magics
            )
        returncode=p.wait_end(magics)
        p.write_contents()
        magics['status']=''
        if returncode != 0:  # Compilation failed
            self.mymagics._logln(''.join((str(s) for s in ccmd)),3)
            self.mymagics._logln("Javac exited with code {}, the executable will not be executed".format(returncode),3)

            # delete source files before exit
            # os.remove(source_filename)
            # os.remove(binary_file.name)
        return p.returncode,outfile


[MyPythonKernel125439] Info:src/j_exec_javac_.py
[MyPythonKernel125439] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/Java/src/j_exec_javac_.py created successfully
